In [1]:
import pandas as pd
import numpy as np  
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
import pylab as plt

In [2]:
X_val = np.load('X_valid.npy')
Y_val = np.load('Y_valid.npy')
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

In [3]:
X_train.shape

(2700, 64, 96, 12)

In [4]:
X_train = np.reshape(X_train, (2700,12,64,96,1))


In [5]:

def create_convlstm_model(input_shape):
    x = keras.layers.Input(shape = input_shape)
    
    y = keras.layers.ConvLSTM2D(12, (3,3), activation='relu', padding = 'same',return_sequences=True)(x)
    y = keras.layers.MaxPooling3D(pool_size=(1, 4, 4))(y)

    y = keras.layers.ConvLSTM2D(24, (3,3), activation='relu', padding = 'same',return_sequences=True)(y)
    y = keras.layers.MaxPooling3D(pool_size=(1, 4, 4))(y)

    y = keras.layers.ConvLSTM2D(24, (3,3), activation='relu', padding = 'same',return_sequences=False)(y)
    
    y = keras.layers.GlobalAveragePooling2D()(y)

#    y = keras.layers.Dropout(0.5)
    
    y = keras.layers.Dense(12) (y)    # add final output layer with a softmax activation
    model = keras.models.Model(inputs=x, outputs=y)
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['MeanSquaredError'])
    
    return model

In [6]:
CONVLSTM = create_convlstm_model(input_shape = (12,64,96,1))
# take a look at how the image tensor changes shape as it passes through the CNN:
CONVLSTM.summary()


2022-08-18 16:08:45.490615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 64, 96, 1)]   0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 12, 64, 96, 12)    5664      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 12, 16, 24, 12)   0         
 )                                                               
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 12, 16, 24, 24)    31200     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 12, 4, 6, 24)     0         
 3D)                                                             
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 4, 6, 24)          41568 

In [7]:
n_epochs = 10
hist = CONVLSTM.fit(X_train, Y_train, epochs=n_epochs, validation_data=(X_val,Y_val))

Epoch 1/10
39/85 [============>.................] - ETA: 11:43 - loss: 1.5169 - mean_squared_error: 1.5169

KeyboardInterrupt: 